# This file include all the code used to produce simulation results

In [ ]:
# Import packages and define functions
import numpy as np
import tensorflow as tf
import keras as keras
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
import xgboost
import time

d = 15
theta = np.hstack((np.array([2, 1.5, 1]), np.zeros(d-3)))
width = 15

def divide0(a, b):
    # 0/0 = 0 if the empirical probability is zero
    return b and a / b or 0

def fpfn(Y_pred, Y_test, G_test):
    fpg1 = divide0(np.sum((Y_pred == 1) & (Y_test == 0) & (G_test == 1)), np.sum((Y_test == 0) & (G_test == 1)))
    fpg0 = divide0(np.sum((Y_pred == 1) & (Y_test == 0) & (G_test == 0)), np.sum((Y_test == 0) & (G_test == 0)))    
    fng1 = divide0(np.sum((Y_pred == 0) & (Y_test == 1) & (G_test == 1)), np.sum((Y_test == 1) & (G_test == 1)))
    fng0 = divide0(np.sum((Y_pred == 0) & (Y_test == 1) & (G_test == 0)), np.sum((Y_test == 1) & (G_test == 0)))
    return [[fpg0, fng0], [fpg1, fng1]]

def index_compute(G, Z, tau):
    return G + np.dot(Z, theta) + tau * (np.sum(np.square(Z), axis = 1)/d + 2*Z[:, 0]*np.sum(Z[:, 1:], axis = 1))

def classification(tau, rho, n, W, M):   
    nt = int(0.7*n) # 70/30 split in traning/test samples
    logit = LogisticRegression(penalty = None)
    lasso_grid = [{'C': np.logspace(-3, 1, 5)}]
    lasso = GridSearchCV(LogisticRegression(penalty = 'l1', solver = "liblinear"), lasso_grid, cv=5)
    svm_grid = [{'C': np.logspace(-1, 1, 5)}]
    svm = GridSearchCV(SVC(), svm_grid, cv=5)
    xgb_grid = [{'n_estimators': [50, 100, 150, 200, 250]}]
    xgb = GridSearchCV(xgboost.XGBClassifier(use_label_encoder=False, eval_metric = 'error'), xgb_grid, cv=5)
    poly = PolynomialFeatures(degree = 3)
    
    sl = keras.models.Sequential([
        keras.Input(shape=[d+1]),
        keras.layers.Dense(W, activation='sigmoid'),
        keras.layers.Dense(2, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])
    sl.compile(optimizer='adam', loss='hinge')
    dl = keras.models.Sequential([
        keras.Input(shape=[d+1]),
        keras.layers.Dense(W, activation='relu'),
        keras.layers.Dense(W, activation='relu'),
        keras.layers.Dense(W, activation='relu'),
        keras.layers.Dense(W, activation='relu'),
        keras.layers.Dense(W, activation='relu'),
        keras.layers.Dense(2, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])
    dl.compile(optimizer='adam', loss='hinge')

    error_l = np.zeros(1)
    error_l1 = np.zeros(1)
    error_svm = np.zeros(1)
    error_xgb = np.zeros(1)
    error_sl = np.zeros(1)
    error_dl = np.zeros(1)
    errors = np.zeros([12, 2])

    for m in range(M):
        G = np.random.binomial(n=1, p=rho, size=n)
        Z = np.random.normal(size = (n, d))
        eps = np.random.normal(size = (n, 1))
        X = np.c_[G, Z]
        index = index_compute(G, Z, tau)
        Y = index >= eps[:, 0]
        
        X_train = X[:nt, :]
        X_test = X[nt:, :]
        Y_train = Y[:nt]
        Y_test = Y[nt:]
        G_test = G[nt:]
        
        logit.fit(X_train, Y_train)        
        Y_l = logit.predict(X_test).reshape(-1)
        error_l = error_l + np.mean(Y_l != Y_test) / M

        X_poly = poly.fit_transform(Z)[:, 1:]
        X_train_poly = X_poly[:nt, :]
        X_test_poly = X_poly[nt:, :]
        lasso.fit(X_train_poly, Y_train)
        Y_l1 = lasso.predict(X_test_poly).reshape(-1)
        error_l1 = error_l1 + np.mean(Y_l1 != Y_test) / M
        
        svm.fit(X_train, Y_train)
        Y_svm = svm.predict(X_test).reshape(-1)
        error_svm = error_svm + np.mean(Y_svm != Y_test) / M

        xgb.fit(X_train, Y_train)
        Y_xgb = xgb.predict(X_test).reshape(-1)
        error_xgb = error_xgb + np.mean(Y_xgb != Y_test) / M
        
        sl.fit(X_train, Y_train, epochs = 30, verbose = 0)
        Y_sl = np.sign(sl.predict(X_test).reshape(-1)) == 1
        error_sl = error_sl + np.mean(Y_sl != Y_test) / M
        
        dl.fit(X_train, Y_train, epochs = 30, verbose = 0)
        Y_dl = np.sign(dl.predict(X_test).reshape(-1)) == 1
        error_dl = error_dl + np.mean(Y_dl != Y_test) / M

        errors = errors + np.concatenate((fpfn(Y_l, Y_test, G_test),
                                          fpfn(Y_l1, Y_test, G_test),
                                          fpfn(Y_xgb, Y_test, G_test),
                                          fpfn(Y_svm, Y_test, G_test),
                                          fpfn(Y_sl, Y_test, G_test),
                                          fpfn(Y_dl, Y_test, G_test))) / M
        
    misclass_groups = pd.DataFrame(errors, #/ np.tile([[1-rho, 1-rho], [rho, rho]], (6, 1)) / (n-nt),
                               columns=['FP', 'FN'],
                               index = ['Logit', '',
                                        'LASSO', '',
                                        'XGB', '',
                                        'SVM', '',
                                        'SL', '',
                                        'DL', ''])
    misclass_groups['Total'] = [np.round(error_l[0], 2), '',
                                np.round(error_l1[0], 2), '',
                                np.round(error_xgb[0], 2), '',
                                np.round(error_svm[0], 2), '',
                                np.round(error_sl[0], 2), '',
                                np.round(error_dl[0], 2), '']
    return misclass_groups

In [ ]:
# Table 1: Monte Carlo Simulation Results: symmetric classification
# Sample size n = 1,000
np.random.seed(2024)
tf.random.set_seed(2024)
experiments = 100
n = 1000
t0 = time.time()
tab1 = classification(tau = 1, rho = 0.2, n = n, W = width, M = experiments)
tab2 = classification(tau = 1, rho = 0.5, n = n, W = width, M = experiments)
tab3 = classification(tau = 0, rho = 0.2, n = n, W = width, M = experiments)
tab4 = classification(tau = 0, rho = 0.5, n = n, W = width, M = experiments)
group = pd.DataFrame(np.tile([0, 1], 6), tab1.index)
tab = pd.concat((group, tab1, tab2, tab3, tab4), axis = 1)
print(round(tab, 2))

t1 = time.time()
print("Time elapsed: ", t1-t0)

print(np.round(tab, 2).to_latex())


In [ ]:
# Table 1: Monte Carlo Simulation Results: symmetric classification
# Sample size n = 10,000
np.random.seed(2024)
tf.random.set_seed(2024)
experiments = 10
n = 10000
width = 15
t0 = time.time()
tab1 = classification(tau = 1, rho = 0.2, n = n, W = width, M = experiments)
tab2 = classification(tau = 1, rho = 0.5, n = n, W = width, M = experiments)
tab3 = classification(tau = 0, rho = 0.2, n = n, W = width, M = experiments)
tab4 = classification(tau = 0, rho = 0.5, n = n, W = width, M = experiments)
group = pd.DataFrame(np.tile([0, 1], 6), tab1.index)
tab = pd.concat((group, tab1, tab2, tab3, tab4), axis = 1)
print(round(tab, 2))

t1 = time.time()
print("Time elapsed: ", t1-t0)

print(np.round(tab, 2).to_latex())


In [ ]:
# Table 2: Monte Carlo Simulation Results: symmetric vs. asymmetric Logit
np.random.seed(2024)
def logit(phi1, phi0, psi1, psi0, rho, tau, n, M):
    nt = int(0.7*n)
    logit = LogisticRegression(solver = "liblinear")
    wlogit = LogisticRegression(solver = "liblinear")
    ll = np.zeros(M)
    wl = np.zeros(M)
    theta = np.hstack((np.array([1, 0.9, 0.8]), np.zeros(d-3)))

    for m in range(M):
        G = np.random.binomial(n=1, p=rho, size=n)
        Z = np.random.normal(size = (n, d))
        eps = np.random.normal(size = (n, 1))
        X = np.c_[G, Z]
        index = index_compute(G, Z, tau)
        Y = index >= 0.1*eps[:, 0]
        
        X_train = X[:nt, :]
        X_test = X[nt:, :]
        Y_train = Y[:nt]
        Y_test = Y[nt:]
        G_test = G[nt:]

        w = (2*Y-1)*((psi1 - phi1)*G + (psi0 - phi0)*(1-G)) + (psi1 + phi1)*G + (psi0 + phi0)*(1-G)
        w_train = w[:nt]

        logit.fit(X_train, Y_train)
        Y_l = logit.predict(X_test).reshape(-1)
        wlogit.fit(X_train, Y_train, sample_weight = w_train)        
        Y_wl = wlogit.predict(X_test).reshape(-1)
        ll[m] = np.mean((psi1 * G_test + psi0 * (1 - G_test)) * ((Y_test == 1) & (Y_l == 0))
                             + (phi1 * G_test + phi0 * (1 - G_test)) * ((Y_test == 0) & (Y_l == 1)))
        wl[m] = np.mean((psi1 * G_test + psi0 * (1 - G_test)) * ((Y_test == 1) & (Y_wl == 0))
                              + (phi1 * G_test + phi0 * (1 - G_test)) * ((Y_test == 0) & (Y_wl == 1)))

    pr = np.mean(ll > wl)
    summary = pd.DataFrame(ll/wl*(wl>0)).describe()
    return pr, summary



params = [[1, 1.65, 1, 3, 0.2, 0],
            [1, 1.65, 1, 3, 0.5, 0],
            [1, 1.65, 1, 3, 0.2, 1],
            [1, 2, 1, 3, 0.2, 0],
            [1, 1.65, 2, 3, 0.2, 0],
            [1, 1.65, 3, 3, 0.2, 0],
            [1, 1.65, 1, 4, 0.2, 0],
            [1, 1.65, 2, 3, 0.2, 0],
            [1, 1.65, 3, 3, 0.2, 0]]

pr = np.zeros(len(params))
summary = np.zeros((5, len(params)))

t0 = time.time()
M = 5000
# n = 1,000
for i in range(len(params)):
               pr[i], stats = logit(phi1=params[i][0], phi0=params[i][1],
                                   psi1=params[i][2], psi0=params[i][3], rho=params[i][4],
                                   tau = params[i][5], n = 1000, M = M)
               summary[:, i] = np.array(stats)[3:].reshape(-1)
pr1000 = np.round(pr, 2)         
print(pd.DataFrame.to_latex(pd.DataFrame(np.round(summary, 2))))

# n = 5,0000
for i in range(len(params)):
               pr[i], stats = logit(phi1=params[i][0], phi0=params[i][1],
                                   psi1=params[i][2], psi0=params[i][3], rho=params[i][4],
                                   tau = params[i][5], n = 5000, M = M)
               summary[:, i] = np.array(stats)[3:].reshape(-1)
pr5000 = np.round(pr, 2)         
print(pd.DataFrame.to_latex(pd.DataFrame(np.round(summary, 2))))

# n = 10,0000
for i in range(len(params)):
               pr[i], stats = logit(phi1=params[i][0], phi0=params[i][1],
                                   psi1=params[i][2], psi0=params[i][3], rho=params[i][4],
                                   tau = params[i][5], n = 10000, M = M)
               summary[:, i] = np.array(stats)[3:].reshape(-1)
pr10000 = np.round(pr, 2)         
print(pd.DataFrame.to_latex(pd.DataFrame(np.round(summary, 2))))

# Print Table 2
print(pd.DataFrame.to_latex(pd.DataFrame(np.vstack((pr1000,pr5000,pr10000)), index=['1,000', '5,000', '10,000'])))

t1 = time.time()
print("Time elapsed: ", t1-t0)


In [ ]:
# Table 3: Monte Carlo Simulation Results: plug-in vs. asymmetric Logit
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import time

np.random.seed(2024)

def logit(phi1, phi0, psi1, psi0, rho, tau, n, M):
    nt = int(0.7*n)
    logit = LogisticRegression(solver = "liblinear")
    wlogit = LogisticRegression(solver = "liblinear")
    ll = np.zeros(M)
    wl = np.zeros(M)
    pl = np.zeros(M)
    theta = np.hstack((np.array([1, 0.9, 0.8]), np.zeros(d-3)))

    for m in range(M):
        G = np.random.binomial(n=1, p=rho, size=n)
        Z = np.random.normal(size = (n, d))
        eps = np.random.normal(size = (n, 1))
        X = np.c_[G, Z]
        index = index_compute(G, Z, tau)
        Y = index >= 0.1*eps[:, 0]
        
        X_train = X[:nt, :]
        X_test = X[nt:, :]
        Y_train = Y[:nt]
        Y_test = Y[nt:]
        G_test = G[nt:]

        w = (2*Y-1)*((psi1 - phi1)*G + (psi0 - phi0)*(1-G)) + (psi1 + phi1)*G + (psi0 + phi0)*(1-G)
        w_train = w[:nt]

        logit.fit(X_train, Y_train)
        #Y_l = logit.predict(X_test).reshape(-1)
        wlogit.fit(X_train, Y_train, sample_weight = w_train)        
        Y_wl = wlogit.predict(X_test).reshape(-1)
        c = G_test * phi1/(phi1+psi1) + (1-G_test) * phi0/(phi0+psi0)
        Y_p = 2*(logit.predict_proba(X_test)[:, 1] > c) - 1

        wl[m] = np.mean((psi1 * G_test + psi0 * (1 - G_test)) * ((Y_test == 1) & (Y_wl == 0))
                        + (phi1 * G_test + phi0 * (1 - G_test)) * ((Y_test == 0) & (Y_wl == 1)))
        pl[m] = np.mean((psi1 * G_test + psi0 * (1 - G_test)) * ((Y_test == 1) & (Y_p == 0))
                        + (phi1 * G_test + phi0 * (1 - G_test)) * ((Y_test == 0) & (Y_p == 1)))

    pr = np.mean(pl > wl)
    summary = pd.DataFrame(pl/wl*(wl>0)).describe()
    return pr, summary


params = [[1, 1.65, 1, 3, 0.2, 0],
            [1, 1.65, 1, 3, 0.5, 0],
            [1, 1.65, 1, 3, 0.2, 1],
            [1, 2, 1, 3, 0.2, 0],
            [1, 1.65, 2, 3, 0.2, 0],
            [1, 1.65, 3, 3, 0.2, 0],
            [1, 1.65, 1, 4, 0.2, 0],
            [1, 1.65, 2, 3, 0.2, 0],
            [1, 1.65, 3, 3, 0.2, 0]]

pr = np.zeros(len(params))
summary = np.zeros((5, len(params)))

t0 = time.time()
M = 5000
# n = 1,000
for i in range(len(params)):
               pr[i], stats = logit(phi1=params[i][0], phi0=params[i][1],
                                   psi1=params[i][2], psi0=params[i][3], rho=params[i][4],
                                   tau = params[i][5], n = 1000, M = M)
               summary[:, i] = np.array(stats)[3:].reshape(-1)
pr1000 = np.round(pr, 2)         
print(pd.DataFrame.to_latex(pd.DataFrame(np.round(summary, 2))))

# n = 5,0000
for i in range(len(params)):
               pr[i], stats = logit(phi1=params[i][0], phi0=params[i][1],
                                   psi1=params[i][2], psi0=params[i][3], rho=params[i][4],
                                   tau = params[i][5], n = 5000, M = M)
               summary[:, i] = np.array(stats)[3:].reshape(-1)
pr5000 = np.round(pr, 2)         
print(pd.DataFrame.to_latex(pd.DataFrame(np.round(summary, 2))))

# n = 10,0000
for i in range(len(params)):
               pr[i], stats = logit(phi1=params[i][0], phi0=params[i][1],
                                   psi1=params[i][2], psi0=params[i][3], rho=params[i][4],
                                   tau = params[i][5], n = 10000, M = M)
               summary[:, i] = np.array(stats)[3:].reshape(-1)
pr10000 = np.round(pr, 2)         
print(pd.DataFrame.to_latex(pd.DataFrame(np.round(summary, 2))))

# Print Table 3
print(pd.DataFrame.to_latex(pd.DataFrame(np.vstack((pr1000,pr5000,pr10000)), index=['1,000', '5,000', '10,000'])))

t1 = time.time()
print("Time elapsed: ", t1-t0)

In [ ]:
# Figure 3: Asymmetric Binary Choice
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import xgboost
import time

np.random.seed(2024)
tf.random.set_seed(2024)

def logit(phi1, phi0, psi1, psi0, tau, n, M, erm):
    rho = 0.2
    nt = int(0.7*n)
    logit = LogisticRegression(solver = "liblinear")
    total = 0
    errors = np.zeros([2, 2])
    theta = np.hstack((np.array([2, 1.5, 1]), np.zeros(d-3)))

    for m in range(M):
        G = np.random.binomial(n=1, p=rho, size=n)
        Z = np.random.normal(size = (n, d))
        eps = np.random.normal(size = (n, 1))
        X = np.c_[G, Z]
        index = index_compute(G, Z, tau)
        Y = index >= eps[:, 0]
        
        X_train = X[:nt, :]
        X_test = X[nt:, :]
        Y_train = Y[:nt]
        Y_test = Y[nt:]
        G_test = G[nt:]

        w = (2*Y-1)*((psi1 - phi1)*G + (psi0 - phi0)*(1-G)) + (psi1 + phi1)*G + (psi0 + phi0)*(1-G)
        w_train = w[:nt]
        
        logit.fit(X_train, Y_train, sample_weight = w_train)
        if erm == 1:
            Y_l = logit.predict(X_test).reshape(-1)
        else:
            c = G_test * phi1/(phi1+psi1) + (1-G_test) * phi0/(phi0+psi0)
            Y_l = logit.predict_proba(X_test)[:, 1] > c
            
        total = total + np.mean(Y_l != Y_test) / M
        errors = errors + fpfn(Y_l, Y_test, G_test)   
    return total, errors / M

def gb(phi1, phi0, psi1, psi0, tau, n, M, erm):
    rho = 0.2
    nt = int(0.7*n)
    #xgb_grid = [{'n_estimators': [50, 100, 150, 200, 250]}]
    #xgb = GridSearchCV(xgboost.XGBClassifier(), xgb_grid, cv=10)
    xgb = xgboost.XGBClassifier(n_estimators = 150, eval_metric = 'error')

    total = np.zeros(1)
    errors = np.zeros([2, 2])
    theta = np.hstack((np.array([2, 1, 0.9, 0.8]), np.zeros(d-3)))

    for m in range(M):
        G = np.random.binomial(n=1, p=rho, size=n)
        Z = np.random.normal(size = (n, d))
        eps = np.random.normal(size = (n, 1))
        X = np.c_[G, Z]
        index = index_compute(G, Z, tau)
        Y = index >= eps[:, 0]
        
        X_train = X[:nt, :]
        X_test = X[nt:, :]
        Y_train = Y[:nt]
        Y_test = Y[nt:]
        G_test = G[nt:]

        w = (2*Y-1)*((psi1 - phi1)*G + (psi0 - phi0)*(1-G)) + (psi1 + phi1)*G + (psi0 + phi0)*(1-G)
        w_train = w[:nt]
        
        xgb.fit(X_train, Y_train, sample_weight = w_train)
        if erm == 1:
            Y_gb = xgb.predict(X_test).reshape(-1)
        else:
            c = G_test * phi1/(phi1+psi1) + (1-G_test) * phi0/(phi0+psi0)
            Y_gb = xgb.predict_proba(X_test)[:, 1] > c
                   
        total = total + np.mean(Y_gb != Y_test) / M
        errors = errors + fpfn(Y_gb, Y_test, G_test)   
    return total, errors / M

experiments = 5000
sample_size = 1000
grid_length = 30
errors = np.empty([2, 2, grid_length])
total = np.empty(grid_length)

# Logit: false positive rates
t0 = time.time()
grid = np.linspace(1, 1.8, grid_length)
for i in range(grid_length):
    total[i], errors[:, :, i] = logit(phi1 = grid[i], phi0 = 1,
                                   psi1 = 1, psi0 = 1, tau = 0,
                                   n = sample_size, M = experiments,
                                   erm = 1)
    
f = plt.figure()   
plt.plot(grid, errors[0, 0, :], 'b',
         grid, errors[1, 0, :], 'b--',
         grid, errors[0, 1, :], 'r',
         grid, errors[1, 1, :], 'r--')
plt.legend(['FP for G=0', 'FP for G=1', 'FN for G=0', 'FN for G=1'])
plt.xlabel(r'$\varphi_0$')
plt.ylabel('Probability')
plt.show()
f.savefig("logit_phi0.pdf", bbox_inches='tight')

# Logit: false negative rates
grid = np.linspace(1, 1.8, grid_length)
for i in range(grid_length):
    total[i], errors[:, :, i] = logit(phi1 = 1, phi0 = 1,
                                   psi1 = 1, psi0 = grid[i], tau = 0,
                                   n = sample_size, M = experiments,
                                   erm = 1)
    
f = plt.figure()   
plt.plot(grid, errors[0, 0, :], 'b',
         grid, errors[1, 0, :], 'b--',
         grid, errors[0, 1, :], 'r',
         grid, errors[1, 1, :], 'r--')
plt.legend(['FP for G=0', 'FP for G=1', 'FN for G=0', 'FN for G=1'])
plt.xlabel(r'$\psi_0$')
plt.ylabel('Probability')
plt.show()
f.savefig("logit_psi0.pdf", bbox_inches='tight')

t1 = time.time()
print("Time elapsed: ", t1-t0)

# Boosting: false positive rates
t0 = time.time()
grid = np.linspace(1, 3, grid_length)
for i in range(grid_length):
    total[i], errors[:, :, i] = gb(phi1 = 1, phi0 = grid[i],
                                   psi1 = 1, psi0 = 1, tau = 0,
                                   n = sample_size, M = experiments,
                                   erm = 1)
    
f = plt.figure()   
plt.plot(grid, errors[0, 0, :], 'b',
         grid, errors[1, 0, :], 'b--',
         grid, errors[0, 1, :], 'r',
         grid, errors[1, 1, :], 'r--')
plt.legend(['FP for G=0', 'FP for G=1', 'FN for G=0', 'FN for G=1'])
plt.xlabel(r'$\varphi_0$')
plt.ylabel('Probability')
plt.show()
f.savefig("boosting_phi0.pdf", bbox_inches='tight')

# Logit: false negative rates
grid = np.linspace(1, 3, grid_length)
for i in range(grid_length):
    total[i], errors[:, :, i] = gb(phi1 = 1, phi0 = 1,
                                   psi1 = grid[i], psi0 = 1, tau = 0,
                                   n = sample_size, M = experiments,
                                   erm = 1)
    
f = plt.figure()   
plt.plot(grid, errors[0, 0, :], 'b',
         grid, errors[1, 0, :], 'b--',
         grid, errors[0, 1, :], 'r',
         grid, errors[1, 1, :], 'r--')
plt.legend(['FP for G=0', 'FP for G=1', 'FN for G=0', 'FN for G=1'])
plt.xlabel(r'$\psi_0$')
plt.ylabel('Probability')
plt.show()
f.savefig("boosting_psi0.pdf", bbox_inches='tight')

t1 = time.time()
print("Time elapsed: ", t1-t0)